<a href="https://colab.research.google.com/github/SBShimm/Aiffel/blob/master/exploration/Exploration6_SB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작사가를 만들어보자!

In [1]:
import tensorflow as tf
import glob, re
import numpy as np


print(tf.__version__)

2.8.2


## 1. 데이터 불러오기  
준비되어 있는 노래 가사들이 적혀있는 Text File을 불러옵니다.  
불러온 Text File은 Line 단위로 split하여 raw_corpus에 저장해 줍니다.

In [2]:
txt_file_path = '/content/drive/MyDrive/Colab/Datasets/lyrics/*.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


## 2. 데이터 전처리
가사에는 누구의 part인지 (누가 그 부분을 불렀는지) 표시하는 부분이 있습니다. 예를들면 (Kanye: 같은 것) 그런 문장과 빈 문장을 제외하도록 하겠습니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue 

    if idx > 9: break 
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


이제 모델이 문장을 기준을 가지고 잘 학습하기 위해서 토큰화를 해주도록 하겠습니다. 토큰화를 하게 되면 각 단어에 대한 사전이 생성되고 문장의 요소들을 해당 사전에 대한 index로 매핑하여 사용된다. 라고 보면 될 것같습니다.  
일단 토큰화에 대한 전처리를 해줍니다. 단어에 특수문자가 포함되거나 대소문자가 구분되어 다른 단어로 인식되지 않게 하고 문장 앞 뒤에 \<start>와 \<end>를 추가합시다.

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

전처리를 마친 문장을 토큰화 하도록 하겠습니다. 여기서 num_words는 사전에 들어갈 단어의 개수라고 보면 됩니다.  
토큰화를 진행할 때 각 문장의 길이가 다 다르기 때문에 길이가 짧은 문장은 비는 부분이 0으로 자동으로 패딩됩니다.  
너무 긴 문장이 있으면 학습할 가중치가 늘어날테니 maxlen을 설정해 줍시다.  
maxlen을 설정하면 기본값으로는 해당 길이만큼의 단어만 토큰화하고 뒷부분은 잘립니다.  
다른 파라미터를 조정하여 앞부분을 자를 수도 있지만, 여기서는 그냥 사용하겠습니다.

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  36   7 ...   0   0   0]
 [  2  13 440 ...  10  12   3]
 [  2  26  17 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f365f4fd710>


In [7]:
len(tensor[0])

15

이제 source와 target data를 분리하겠습니다.  

In [8]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0]


분리한 소스 데이터와 타겟 데이터를 활용하여 train, test 데이터 셋을 나누어줍니다.

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=1004)

그리고 train data를 이용하여 dataset 객체를 생성해 줍시다. 

In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 14), dtype=tf.int32, name=None), TensorSpec(shape=(128, 14), dtype=tf.int32, name=None))>

## 3. 모델 학습하기
이제 학습을 할차례입니다.  
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되는 모델을 사용하도록 하겠습니다.

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [12]:
embedding_size = 256
hidden_size = 1024
epochs = 20

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/20
550/550 [==============================] - 64s 100ms/step - loss: 3.7000 - val_loss: 3.3286
Epoch 2/20
550/550 [==============================] - 54s 99ms/step - loss: 3.2152 - val_loss: 3.1340
Epoch 3/20
550/550 [==============================] - 54s 99ms/step - loss: 3.0344 - val_loss: 3.0102
Epoch 4/20
550/550 [==============================] - 54s 99ms/step - loss: 2.8976 - val_loss: 2.9396
Epoch 5/20
550/550 [==============================] - 54s 99ms/step - loss: 2.7820 - val_loss: 2.8543
Epoch 6/20
550/550 [==============================] - 54s 99ms/step - loss: 2.6753 - val_loss: 2.7983
Epoch 7/20
550/550 [==============================] - 54s 99ms/step - loss: 2.5760 - val_loss: 2.7464
Epoch 8/20
550/550 [==============================] - 54s 99ms/step - loss: 2.4820 - val_loss: 2.7078
Epoch 9/20
550/550 [==============================] - 54s 99ms/step - loss: 2.3926 - val_loss: 2.6721
Epoch 10/20
550/550 [==============================] - 54s 99ms/step - loss: 2.30

## 4. 하이퍼 파라미터 튜닝
2.5 즈음에서 더이상 학습이 안되네요. 하이퍼 파라미터를 튜닝하여 더 좋은 결과를 내보도록 합시다.
embedding_size는 각 벡터공간에서 단어의 추상적인 표현을 할 수 있는 크기입니다.   
hidden_size는 얼마나 많은 일꾼을 사용할 지 입니다.  
목표는 10에폭 안에 val_loss를 2.2 아래로 떨어뜨리는 것입니다.

In [13]:
embedding_size = 20
hidden_size = 2048
epochs = 10

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(dataset,
                    epochs=epochs,
                    validation_data=(enc_val, dec_val),
                    verbose=1)

Epoch 1/10
1098/1098 [==============================] - 171s 143ms/step - loss: 3.6164 - val_loss: 3.1956
Epoch 2/10
1098/1098 [==============================] - 156s 142ms/step - loss: 3.0355 - val_loss: 2.9467
Epoch 3/10
1098/1098 [==============================] - 156s 142ms/step - loss: 2.7414 - val_loss: 2.7821
Epoch 4/10
1098/1098 [==============================] - 156s 142ms/step - loss: 2.4559 - val_loss: 2.6568
Epoch 5/10
1098/1098 [==============================] - 156s 142ms/step - loss: 2.1797 - val_loss: 2.5696
Epoch 6/10
1098/1098 [==============================] - 156s 142ms/step - loss: 1.9247 - val_loss: 2.5079
Epoch 7/10
1098/1098 [==============================] - 156s 142ms/step - loss: 1.7049 - val_loss: 2.4798
Epoch 8/10
1098/1098 [==============================] - 156s 142ms/step - loss: 1.5203 - val_loss: 2.4718
Epoch 9/10
1098/1098 [==============================] - 156s 142ms/step - loss: 1.3724 - val_loss: 2.4834
Epoch 10/10
1098/1098 [=======================

2.48로 줄기는 했지만 조금더 좋은 결과를 원합니다.  
수많은 시도 결과 토큰화 단계에서 설정했던 maxlen을 20으로 설정하고 첫번째 LSTM Layer에 Dropout을 추가한 것이 결과가 제일 좋았습니다.

In [14]:
def tokenize2(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize2(corpus)

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=1004)

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 100
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  36   7 ...   0   0   0]
 [  2  13 440 ...   0   0   0]
 [  2  26  17 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f36500b4310>
[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


<BatchDataset element_spec=(TensorSpec(shape=(100, 19), dtype=tf.int32, name=None), TensorSpec(shape=(100, 19), dtype=tf.int32, name=None))>

In [15]:
class TextGenerator2(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size_1, hidden_size_2):
        super(TextGenerator2, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size_1, return_sequences=True, dropout=0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size_2, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [16]:
embedding_size = 70
hidden_size_1 = 2000
hidden_size_2 = 2000
epochs = 10

model = TextGenerator2(tokenizer.num_words + 1, embedding_size , hidden_size_1, hidden_size_2)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(dataset,
                    epochs=epochs,
                    validation_data=(enc_val, dec_val),
                    verbose=1)

Epoch 1/10
1405/1405 [==============================] - 240s 163ms/step - loss: 2.7244 - val_loss: 2.4182
Epoch 2/10
1405/1405 [==============================] - 229s 163ms/step - loss: 2.3090 - val_loss: 2.2326
Epoch 3/10
1405/1405 [==============================] - 229s 163ms/step - loss: 2.0872 - val_loss: 2.1095
Epoch 4/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.8777 - val_loss: 2.0242
Epoch 5/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.6831 - val_loss: 1.9617
Epoch 6/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.5087 - val_loss: 1.9247
Epoch 7/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.3593 - val_loss: 1.9062
Epoch 8/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.2363 - val_loss: 1.9034
Epoch 9/10
1405/1405 [==============================] - 229s 163ms/step - loss: 1.1351 - val_loss: 1.9140
Epoch 10/10
1405/1405 [=======================

loss가 아주 낮아졌다.  

## 5. 가사 작사해보기
이제 이 학습모델로 가사를 지어보겠습니다.

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

아주 간단한 문장이... 나왔다

In [25]:
generate_text(model, tokenizer, init_sentence="<start> fresh", max_len=20)

'<start> fresh out da dealership crackin up wit cigarsin <end> '

몇가지로 작사를 해봤는데 대부분 학습 데이터의 가사를 가져오는 것 같다.  
검색해보니 이미 있는 노래의 가사인 경우가 많았다.

## 6.회고
1. 한번 돌리는데 너무 오래걸려서 너무 힘들었다 일주일 내내 이거 학습만 한듯..
2. 일주일 내내 하면서 2.203까지 봤는데 maxlen을 20으로 늘리니까 1.9까지 내려가서 약간의 허탈함이 느껴졌다.
3. 임베딩 사이즈를 늘리니까 에폭이 늘어날수록 val_loss가 늘어났다. 임베딩 사이즈가 추상화의 수를 늘리는 거였으니 학습 데이터에 과적합되면서 그런게 아닐까 싶다.